<a href="https://colab.research.google.com/github/sadullahmath/SeaBorn/blob/master/ExtendingPlots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extending Plots with Widgets

In [0]:
import pandas as pd

In [0]:
from bokeh.io import output_notebook
output_notebook()

In [0]:
dataset= pd.read_csv('https://raw.githubusercontent.com/PacktWorkshops/The-Data-Visualization-Workshop/master/Datasets/olympia2016_athletes.csv')

In [4]:
dataset.head(10)

,id,name,nationality,sex,dob,height,weight,sport,gold,silver,bronze
0,736041664,A Jesus Garcia,ESP,male,10/17/69,1.72,64.0,athletics,0,0,0
1,532037425,A Lam Shin,KOR,female,9/23/86,1.68,56.0,fencing,0,0,0
2,435962603,Aaron Brown,CAN,male,5/27/92,1.98,79.0,athletics,0,0,1
3,521041435,Aaron Cook,MDA,male,1/2/91,1.83,80.0,taekwondo,0,0,0
4,33922579,Aaron Gate,NZL,male,11/26/90,1.81,71.0,cycling,0,0,0
5,173071782,Aaron Royle,AUS,male,1/26/90,1.80,67.0,triathlon,0,0,0
6,266237702,Aaron Russell,USA,male,6/4/93,2.05,98.0,volleyball,0,0,1
7,382571888,Aaron Younger,AUS,male,9/25/91,1.93,100.0,aquatics,0,0,0
8,87689776,Aauri Lorena Bokesa,ESP,female,12/14/88,1.80,62.0,athletics,0,0,0
9,997877719,Ababel Yeshaneh,ETH,female,7/22/91,1.65,54.0,athletics,0,0,0


In [0]:
from bokeh.plotting import figure, show, ColumnDataSource
from ipywidgets import interact, widgets

In [6]:
countries = dataset['nationality'].unique()
athletes_per_country = dataset.groupby('nationality').size()
medals_per_country = dataset.groupby('nationality')['gold', 'silver','bronze'].sum()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
max_medals = medals_per_country.sum(axis=1).max()
max_athletes = athletes_per_country.max() 

In [0]:
# setting up the interaction elements
max_athletes_slider=widgets.IntSlider(value=max_athletes, min=0, max=max_athletes, step=1, description='Max. Athletes:', continuous_update=False, orientation='vertical', layout={'width': '100px'})
max_medals_slider=widgets.IntSlider(value=max_medals, min=0, max=max_medals, step=1, description='Max. Medals:', continuous_update=False, orientation='horizontal')

In [0]:
@interact(max_athletes=max_athletes_slider, max_medals=max_medals_slider)
def get_olympia_stats(max_athletes, max_medals):
    show(get_plot(max_athletes, max_medals))

In [0]:
# creating the scatter plot
def get_plot(max_athletes, max_medals):
    filtered_countries=[]
    
    for country in countries:
        if (athletes_per_country[country] <= max_athletes and 
            medals_per_country.loc[country].sum() <= max_medals):
            filtered_countries.append(country)
        
    data_source=get_datasource(filtered_countries)
    TOOLTIPS=[
        ('Country', '@countries'),
        ('Num of Athletes', '@y'),
        ('Gold', '@gold'),
        ('Silver', '@silver'),
        ('Bronze', '@bronze')
    ]
    
    plot=figure(title='Rio Olympics 2016 - Medal comparison', 
                x_axis_label='Number of Medals',  
                y_axis_label='Num of Athletes',
                plot_width=800, 
                plot_height=500,
                tooltips=TOOLTIPS)
    
    plot.circle('x', 'y', source=data_source, size=20, color='color', alpha=0.5) 
    return plot  

In [0]:
# get a 6 digit random hex color to differentiate the countries better
import random
def get_random_color():
    return '#%06x' % random.randint(0, 0xFFFFFF)

In [0]:
# build the datasource
def get_datasource(filtered_countries):
    return ColumnDataSource(data=dict(
        color=[get_random_color() for _ in filtered_countries],
        countries=filtered_countries,
        gold=[medals_per_country.loc[country]['gold'] for country in filtered_countries],
        silver=[medals_per_country.loc[country]['silver'] for country in filtered_countries],
        bronze=[medals_per_country.loc[country]['bronze'] for country in filtered_countries],
        x=[medals_per_country.loc[country].sum() for country in filtered_countries],
        y=[athletes_per_country.loc[country].sum() for country in filtered_countries]